In [ ]:
#export
import random
import numpy as np
from copy import deepcopy
from collections import deque, namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from bounce.utils import state2str, array_in_list, T

In [ ]:
# default_exp agents

In [ ]:
#hide
from nbdev.showdoc import show_doc
%load_ext autoreload
%autoreload 2

# Agents

> Deffinition of the different kind of agents that can be implemented. The agents are the entities that perform the learning process to, ultimately, obtain the optimal set of constraints. 

We consider agents based on different techniques of artificial intelligence: 
- Agents based on reinforcement learning (Q-learning)
- Agents based on blind-search (breadth first search)
- Agents based on Monte-Carlo sampling

## Reinforcement learning

The agents based on reinforcement learning implement a value-based algorithm called Q-learning. More precisely, the agent implemented in this framework is based on [deep double Q-learning](https://ojs.aaai.org/index.php/AAAI/article/view/10295).

In [ ]:
#export
class DQNAgent:
    def __init__(self, model, learning_rate=1e-3, criterion=None, optimizer=None, batch_size=128, 
                 target_update=5, gamma=0.85, eps_0=1, eps_decay=0.999, eps_min=0.1):       
        """Agent based on a deep Q-Network (DQN):
        Input: 
            - model: torch.nn.Module with the DQN model. Dimensions must be consistent
            - criterion: loss criterion (e.g., torch.nn.SmoothL1Loss)
            - optimizer: optimization algorithm (e.g., torch.nn.Adam)
            - eps_0: initial epsilon value for an epsilon-greedy policy
            - eps_decay: exponential decay factor for epsilon in the epsilon-greedy policy
            - eps_min: minimum saturation value for epsilon
            - gamma: future reward discount factor for Q-value estimation"""   
        
        # Model
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self._build_target_net()
        self.target_update = target_update
        
        # Parameters
        self.lr = learning_rate
        self.gamma = gamma    # discount factor
        self.epsilon, self.epsilon_min, self.epsilon_decay = eps_0, eps_min, eps_decay   
        self._get_criterion(criterion)
        self._get_optimizer(optimizer)
        self.batch_size = batch_size
        self.memory = deque(maxlen=10000) # Replay memory
        self.transition = namedtuple('Transition', ('state', 'action', 'bound', 'cost', 'next_state'))
        
    def act(self, state):
        "Given a state, return actions ordered by priority."

        if torch.rand(1) <= self.epsilon:
            return torch.randperm(self.state_size)
        else: 
            with torch.no_grad():
                Q = self.q_values(state)
                return torch.argsort(Q, descending=True)
            
    def q_values(self, state):
        "Returns the Q values of each action given a state."
        state = T(state).reshape(1, self.state_size).to(self.device)
        return self.model(state).squeeze()
             
    def replay(self, env):
        "Learn from past events in the memory."
        batch_size = min(len(self.memory), self.batch_size)
        transitions = random.sample(self.memory, batch_size)
        batch = self.transition(*zip(*transitions))    
        
        state_batch, action_batch = torch.cat(batch.state), torch.cat(batch.action)
        next_states = torch.cat(batch.next_state) 
        bound_batch, cost_batch = torch.cat(batch.bound), torch.cat(batch.cost)
        reward_batch = env.reward_fun(bound_batch, cost_batch)
        
        if torch.isnan(reward_batch).any(): 
            shit_idx = torch.isnan(reward_batch)
            raise ValueError(f"Shit went wrong, got nan in {next_states[shit_idx]}"+
                             f"\nBound {bound_batch[shit_idx]}"+
                             f"\nCost {cost_batch[shit_idx]}"+
                             f"\nReward {reward_batch[shit_idx]}"+
                             f"\nReward batch {reward_batch}")

        # Q-values
        state_action_values = self.model(state_batch).gather(1, action_batch.reshape(batch_size, 1))
        # Expected Q-values
        next_state_values = self.target_net(next_states).max(1)[0].detach()*self.gamma + reward_batch
                
        # Optimize the model
        self.optimizer.zero_grad()
        loss = self.criterion(state_action_values, next_state_values.unsqueeze(1))
        loss.backward()
        self.optimizer.step()    
            
        if self.epsilon > self.epsilon_min: self.epsilon *= self.epsilon_decay
        if self.epsilon < self.epsilon_min: self.epsilon = self.epsilon_min
            
    def memorize(self, state, action, bound, cost, next_state):  
        "Remember a state-action-state-reward transition."
        info = [torch.FloatTensor(state).reshape(1, self.state_size).to(self.device), 
                torch.tensor([action], device = self.device), 
    #                 torch.FloatTensor([reward]).to(self.device), 
                torch.FloatTensor([bound]).to(self.device),
                torch.FloatTensor([cost]).to(self.device),
                torch.FloatTensor(next_state).reshape(1, self.state_size).to(self.device)
                ]
        self.memory.append(self.transition(*info))
            
    def _build_target_net(self):
        model_params = list(self.model.parameters())
        self.state_size = model_params[0].size()[-1]
        self.action_size = model_params[-1].size()[0]
        self.target_net = self.model.__class__(self.state_size, self.action_size).to(self.device)   
        self.target_net.load_state_dict(self.model.state_dict())
        self.target_net.eval()
        
    def _get_criterion(self, criterion=None):
        if criterion is None: self.criterion = nn.SmoothL1Loss(reduction='sum')
        else:                 self.criterion = criterion
            
    def _get_optimizer(self, optimizer=None):
        if optimizer is None: self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        else:                 self.optimizer = optimizer

We provide a default architecture for the neural network that encodes the Q-values, usually referred to as deep Q-Network (DQN). 

In [ ]:
#export
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        self.fc1 = nn.Linear(state_size, 3*state_size)
        self.fc2 = nn.Linear(3*state_size, 2*action_size)
        self.fc3 = nn.Linear(2*action_size, 2*action_size)
        self.fc4 = nn.Linear(2*action_size, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

## Blind-search

The agents based on tree search currently only implement blind-search techniques, such as breadth first search. 

In [ ]:
#export
class BrFSAgent:
    def __init__(self, initial_state):
        "Agent based on Breadth First Search (BrFS)."
        self.state_size = len(initial_state)
        self.open = deque([initial_state])
        self.closed = set()
        
    def expand(self):
        "Expands the first node of the open"
        try :
            state = self.open.popleft()
            self.add_closed(state)
            return state, np.random.permutation(self.state_size)
        except:
            return [], []
    
    def in_open(self, state):
        "Boolean indicating whether state is in open"
        return array_in_list(state, self.open)
    
    def in_closed(self, state):
        "Boolean indicating whether state is in closed"
        return state2str(state) in self.closed
    
    def add_open(self, state): 
        "Adds state to open"
        self.open.append(state)
    
    def add_closed(self, state):
        "Adds state to closed"
        self.closed.update([state2str(state)])

In [ ]:
agent = BrFSAgent(3, np.array([1, 1, 1, 0, 0, 0]))

In [ ]:
agent.expand()

[array([0, 1, 1, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0]),
 array([1, 1, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 0, 0]),
 array([1, 1, 1, 0, 1, 0]),
 array([1, 1, 1, 0, 0, 1])]

## Monte-Carlo

The agents based on Monte-Carlo sampling follow the Metropolis-Hastings algorithm to move between states. A random action (new state) is proposed and the move is accepted or rejected with a certain probability.  

In [ ]:
#export
class MCAgent:
    def __init__(self, beta=0.1):
        self.beta = beta
        self.accepted = 0
    
    def act(self, state):
        "Try random actions."
        return np.random.permutation(len(state))
    
    def accept(self, r1, r2):
        "Boolean indicating whether to accept or not the movement from 1 to 2."
        accept = np.random.random() <= min(1, np.exp((r2-r1)/self.beta))
        self.accepted += accept
        return accept

## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_environment.ipynb.
Converted 01_agents.ipynb.
Converted 02_budget_profiles.ipynb.
Converted 03_hamiltonian.ipynb.
Converted 04_training.ipynb.
Converted 05_utils.ipynb.
Converted 06_sdp.ipynb.
Converted index.ipynb.
